In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
# Định nghĩa hàm Focal Loss
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true * alpha + (K.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (K.ones_like(y_true) - y_true) * (1 - y_pred)
        fl = -alpha_t * K.pow((K.ones_like(y_true) - p_t), gamma) * K.log(p_t)
        return K.mean(K.sum(fl, axis=-1))
    return focal_loss_fixed

In [ ]:
# Đường dẫn dữ liệu (sau khi xóa folder "Nghi ngờ")
train_dir = 'kneeKL224/train/'
test_dir = 'kneeKL224/val/'

# Các thông số đầu vào
input_size = (224, 224)
batch_size = 32
num_classes = 4 # Cập nhật số lớp đầu ra theo dữ liệu
epochs = 50

In [ ]:

# Cải thiện Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2],  # Điều chỉnh độ sáng nhẹ
    horizontal_flip=True,  # Lật ngang
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Thiết lập bộ tạo dữ liệu
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Tính trọng số lớp cho việc huấn luyện không cân bằng
class_labels = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(class_labels), y=class_labels)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:

# Khởi tạo mô hình DenseNet121
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune 40-50 lớp cuối
for layer in base_model.layers[:-50]:  # Đóng băng các lớp trước 50 lớp cuối
    layer.trainable = False
for layer in base_model.layers[-50:]:  # Cho phép trainable 50 lớp cuối
    layer.trainable = True

# Tạo mô hình mới
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
# Cấu hình quá trình huấn luyện với Focal Loss
initial_learning_rate = 0.0007
model.compile(optimizer=optimizers.Adam(learning_rate=initial_learning_rate),
              loss=focal_loss(gamma=2.0, alpha=0.25),
              metrics=['accuracy'])
# Định nghĩa callback
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
]

In [ ]:
# Huấn luyện mô hình
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    class_weight=class_weights_dict,
    callbacks=callbacks
)


In [ ]:
# In độ chính xác tốt nhất từ lịch sử huấn luyện
best_val_accuracy = max(history.history['val_accuracy'])
print(f"Độ chính xác cao nhất trên tập validation: {best_val_accuracy * 100:.2f}%")

# Đánh giá mô hình tốt nhất
best_model = tf.keras.models.load_model(
    'best_model_4_classes.keras',
    custom_objects={'focal_loss_fixed': focal_loss()}
)

val_loss, val_accuracy = best_model.evaluate(test_generator)
print(f"Độ chính xác của mô hình tốt nhất: {val_accuracy * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

# Vẽ đồ thị Accuracy
plt.figure(figsize=(12, 5))

# Đồ thị accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Đồ thị loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()
